# Convolutional Neural Networks
This notebook will guide you through the use of the `keras` package to train convolutional neural networks for handwritten digits classification. You are going to use the `mnist` dataset from LeCun et al. 1998.

## Loading the packages

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.layers import Input, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils import np_utils
from matplotlib import pyplot as pl
from sklearn import metrics as me

from utils import plot_confusion_matrix

%matplotlib inline

First, create some useful functions

In [ ]:
def build_grid_of_images(array):
    assert len(array.shape) == 3
    dim_0 = np.sqrt(array.shape[0])
    assert dim_0.is_integer()

    temp_out = np.reshape(array, (dim_0, dim_0, array.shape[1], array.shape[2]))
    temp_out = np.rollaxis(temp_out, 1, 3)
    return np.reshape(temp_out, (dim_0 * array.shape[1], dim_0 * array.shape[2]))


def plot_conv_layer_output(temp_out, title):
    temp_to_plot = build_grid_of_images(temp_out)
    pl.imshow(temp_to_plot, interpolation='nearest', cmap=pl.get_cmap('Greys'))
    ax = pl.gca()
    ax.set_xticks(np.arange(-0.5, temp_to_plot.shape[0] + 0.5, temp_out.shape[1]))
    ax.set_yticks(np.arange(-0.5, temp_to_plot.shape[0] + 0.5, temp_out.shape[2]))
    pl.grid()
    pl.tick_params(axis='both', which='both', bottom='off', top='off', left='off', right='off', labelbottom='off',
                   labelleft='off')
    pl.title(title)


def plot_dense_layer_output(temp_out, title):
    pl.bar(np.arange(temp_out.shape[1]) - 0.4, temp_out[0, :])
    pl.xlim(-0.5, temp_out.shape[1])
    pl.grid()
    pl.title(title)

Load the `mnist` dataset and normalize in the range [0, 1]

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

n_train, height, width = X_train.shape
n_test, _, _ = X_test.shape

X_train = X_train.reshape(n_train, height, width, 1).astype('float32')
X_test = X_test.reshape(n_test, height, width, 1).astype('float32')

X_train /= 255.0
X_test /= 255.0

n_classes = 10

print(n_train, 'train samples')
print(n_test, 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [ ]:
print('X_train.shape = ', X_train.shape)
print('Y_train.shape = ', Y_train.shape)
print('X_test.shape = ', X_test.shape)
print('Y_test.shape = ', Y_test.shape)

Create the CNN and show its architecture

In [ ]:
model = Sequential([
    Input(shape=(height, width, 1), name='l0'),
    Conv2D(32, kernel_size=(3, 3), activation='relu', name='l1'),
    MaxPooling2D(pool_size=(2, 2), name='l1_mp'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', name='l2'),
    MaxPooling2D(pool_size=(2, 2), name='l2_mp'),
    Flatten(name='flat'),
    Dropout(0.5, name='drop'),
    Dense(n_classes, activation='softmax', name='l3'),
])

model.summary()

Define some constants and train de CNN

In [ ]:
batch_size = 128
n_epoch = 15

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=n_epoch, verbose=1, validation_split=0.1)

422/422 [==============================] - 4s 9ms/step - loss: 0.0807 - accuracy: 0.9751 - val_loss: 0.0552 - val_accuracy: 0.9853
Epoch 4/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0688 - accuracy: 0.9784 - val_loss: 0.0468 - val_accuracy: 0.9863
Epoch 5/15
422/422 [==============================] - 4s 8ms/step - loss: 0.0622 - accuracy: 0.9801 - val_loss: 0.0452 - val_accuracy: 0.9878
Epoch 6/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0572 - accuracy: 0.9826 - val_loss: 0.0379 - val_accuracy: 0.9902
Epoch 7/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0517 - accuracy: 0.9849 - val_loss: 0.0388 - val_accuracy: 0.9893
Epoch 8/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0476 - accuracy: 0.9851 - val_loss: 0.0358 - val_accuracy: 0.9905
Epoch 9/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0447 - accuracy: 0.9863 - val_loss: 0.0334 - val_accuracy: 0.9908
Epoch 10/15
422/4

Show the performance of the model

In [ ]:
pl.plot(history.history['loss'], label='Training')
pl.plot(history.history['val_loss'], label='Testing')
pl.legend()
pl.grid()

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Confusion matrix

In [ ]:
pred = model.predict(X_test)
print('pred.shape = ', pred.shape)
pred = np.argmax(pred, axis=-1)
cm = me.confusion_matrix(y_test, pred)

plot_confusion_matrix(cm, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])